In [1]:
import instruments as inst
import numpy as np

# Step 1: Reading in csv - extracting single diffs and sums and configurations

In [2]:
file_path = "20230914_processed_table.csv"
wavelength_string = "625-50"

interleaved_values, interleaved_stds, configuration_list = inst.read_csv(file_path, obs_filter = wavelength_string)
print("Interleaved Values")
print(interleaved_values[0 : 10])
print("Interleaved Stds")
print(interleaved_stds[0 : 10])
print("Configuration List")
print(configuration_list[0 : 10])

Interleaved Values
[-1080.   1763.5   636.   1712.  -1081.   1759.    758.   1706.  -1312.
  1774. ]
Interleaved Stds
[ 66.22258   81.24302  128.18863   79.7065    66.52477   82.804184
  65.0891    80.72938   71.02066  356.43402 ]
Configuration List
[{'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 45.0}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 45.0}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 57.5}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 57.5}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 70.0}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 70.0}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 82.5}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 82.5}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 95.0}}, {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 95.0}}]


# Step 2: Creating a system Mueller matrix object

In [3]:
# TODO: Change this to be values from last best scipy.minimize fit with old code
# NOTE: Components must be listed downstream to upstream
# Define the instrument configuration as a system dictionary
system_dict = {
    "components": {
        "wollaston": {
            "type": "wollaston_prism_function",
            "properties": {"beam": "o", "transmission_ratio": 1},
        },
        "dichroic": {
            "type": "general_retarder_function",
            "properties": {"phi": 2 * np.pi * 0, "theta": 0},
        },
        "flc": {
            "type": "general_retarder_function",
            "properties": {"phi": 2 * np.pi * 0.5, "theta": 0, "delta_theta": 0},
        },
        "optics": {
            "type": "diattenuator_retarder_function",
            "properties": {"phi": 2 * np.pi * 0, "epsilon": 0},
        },
        "image_rotator": {
            "type": "general_retarder_function",
            "properties": {"phi": 2 * np.pi * 0.5, "theta": 0, "delta_theta": 0},
        },
        "hwp": {
            "type": "general_retarder_function",
            "properties": {"phi": 2 * np.pi * 0.5, "theta": 0, "delta_theta": 0},
        },
        "lp": {
            "type": "general_linear_polarizer_function_with_theta",
            "properties": {"theta": 0},
        },
        # Testing whether generate_system_mueller_matrix can flag this as a non-existent component
        "test_false_type": {
            "type": "test",
            "properties": {"theta": 0, "epsilon": 0}
        },
        # Testing whether generate_system_mueller_matrix can flag epsilon as a
        # non-existent property of rotator
        "test_false_property": {
            "type": "general_retarder_function",
            "properties": {"epsilon": 0}
        }
    }
}

In [4]:
# Converting system dictionary into system Mueller Matrix object

system_mm = inst.generate_system_mueller_matrix(system_dict)
print(system_mm.evaluate())

Error: 'test' is not a valid function in pyMuellerMat.common_mm_functions and will be skipped.
[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


# TEST: Checking parse_configuration & update_system_mm

In [5]:
test_configuration = {
    'lp': {'properties': {'theta': 0.0}},
    'hwp': {'properties': {'theta': 0.0}},
    'image_rotator': {'properties': {'theta': 0.0}},
    'flc': {'properties': {'theta': 0.0}},
    'wollaston': {'properties': {'beam': 'o'}}
    }

values, keywords = inst.parse_configuration(test_configuration)
print("Values: " + str(values))
print("Keywords: " + str(keywords))

updated_system_mm = inst.update_system_mm(values, keywords, system_mm)
# print(updated_system_mm.evaluate())


Values: [{'theta': 0.0}, {'theta': 0.0}, {'theta': 0.0}, {'theta': 0.0}, {'beam': 'o'}]
Keywords: [['lp', 'properties'], ['hwp', 'properties'], ['image_rotator', 'properties'], ['flc', 'properties'], ['wollaston', 'properties']]
Parameter 'properties' not found in component 'lp'. Skipping...
Parameter 'properties' not found in component 'hwp'. Skipping...
Parameter 'properties' not found in component 'image_rotator'. Skipping...
Parameter 'properties' not found in component 'flc'. Skipping...
Parameter 'properties' not found in component 'wollaston'. Skipping...


# Step 4: Creating a dictionary of p0 starting guesses

In [6]:
# Fittin for just the dichroic for now

p0 = {
    "dichroic": {"phi": 0, "theta": 0}
}

# Step 5: Running minimize_system_Mueller_matrix

In [7]:
# Unpolarized light for s_in as there is LP in the system already
s_in = np.array([1, 0, 0, 0])

result = inst.minimize_system_mueller_matrix(p0, system_mm, interleaved_values, 
    interleaved_stds, configuration_list, s_in = s_in)
print(result.x)
print(type(system_mm.evaluate()))      

p0_values:  [0, 0]
p0_keywords:  [['dichroic', 'phi'], ['dichroic', 'theta']]
[0. 0.]
<class 'numpy.ndarray'>
